In [150]:
import csv
import numpy as np
from random import shuffle, sample, choice

In [106]:
fname = 'me_at_the_zoo.in'

In [107]:
ans = open(fname).read().splitlines()

## Parameters:

In [108]:
video_num, endpoint_num, request_num, cache_num, cache_size_limit = map(int, ans[0].split())

In [125]:
video_sizes = list(enumerate(map(int, ans[1].split())))

## Endpoints

In [110]:
rest = ans[2:]
endpoints = []
point = 0
for endpoint in range(endpoint_num):
    end_latent, num_connected = map(int,rest[point].split())
    caches = []
    for l in range(num_connected):
        caches.append(map(int,rest[point+l+1].split()))
    endpoints.append([end_latent, num_connected, caches])
    point += 1 + num_connected

## Requests

In [111]:
rest = rest[point:]

requests = []
for l in range(request_num):
    requests.append(map(int, rest[l].split()))

In [147]:
endpoints

[[1013, 3, [[0, 170], [1, 22], [2, 224]]],
 [696, 2, [[0, 7], [1, 50]]],
 [1114, 3, [[1, 202], [4, 175], [5, 2]]],
 [464, 2, [[1, 24], [8, 25]]],
 [522, 5, [[3, 216], [5, 155], [6, 139], [7, 208], [8, 145]]],
 [321, 4, [[0, 26], [2, 70], [8, 159], [9, 92]]],
 [1288, 2, [[2, 163], [9, 153]]],
 [226, 1, [[7, 86]]],
 [316, 5, [[4, 236], [5, 79], [6, 9], [7, 53], [8, 67]]],
 [365, 5, [[2, 225], [3, 62], [5, 141], [6, 147], [9, 66]]]]

In [122]:
def fill_server(capacity, videos):
    curr = 0
    ret = set()
    indices = list(enumerate(videos))
    shuffle(indices)
    for i, size in indices:
        if curr + size > capacity:
            continue
        ret.add(i)
        curr += size
    return ret

In [139]:
def initial_guess():
    guess = []
    for c in range(cache_num):
        sh = sample(video_sizes, len(video_sizes))
        ans = []
        size = cache_size_limit
        while len(sh) > 0:
            if sh[0][1] < size:
                ans.append(sh[0][0])
                size -= sh[0][1]
            del sh[0]
        guess.append([c,ans])
    return guess

In [142]:
initial_guess()

[[0, [16, 6, 68, 35, 50, 28, 56, 92, 42, 40]],
 [1, [97, 83, 85, 4, 78, 47, 76]],
 [2, [83, 21, 76, 35, 91, 85, 47]],
 [3, [70, 89, 17, 4, 85, 40, 16, 42]],
 [4, [16, 81, 32, 7, 4, 78, 75, 46, 40, 42]],
 [5, [62, 12, 65, 4, 16]],
 [6, [67, 26, 65, 10, 42, 40]],
 [7, [54, 47, 97, 22, 72]],
 [8, [79, 34, 16, 5, 81, 71, 42, 40]],
 [9, [45, 99, 38, 76, 42, 52, 16, 47]]]

In [144]:
mutateRetry = 100
cache_size_limit = 35

In [180]:
def mutate(instance, video_sizes):
    cacheToReplace = choice(instance)
    videoToReplace = choice(cacheToReplace[1])
    for l in range(mutateRetry):
        newVid = choice(video_sizes)[1]
        if newVid not in cacheToReplace and newVid != videoToReplace:
            cacheToReplace[1].remove(videoToReplace)
            cacheToReplace[1].append(newVid)
            if sum([l[1] for l in video_sizes]) <= cache_size_limit:
                return
            cacheToReplace[1].remove(newVid)
            cacheToReplace[1].append(videoToReplace)

## Output

In [ ]:
def write_out(fout='output.in', cache_list):
    with open(fout, 'w') as f:
        f.write(len(cache_list))
